In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [22]:
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('s9_final-2.csv')

In [5]:
df[0:1].T

,0
Unnamed: 0,0
raceId,4
battleTag,Flexitus#21593
overallScore.UNIT_SCORE,9240
overallScore.HERO_SCORE,6580
overallScore.RESOURCE_SCORE,6255
overallScore.TOTAL_SCORE,22075
unitScore.UNITS_PRODUCED,26
unitScore.UNITS_KILLED,15
unitScore.STRUCTURES_PRODUCED,9


In [7]:
# adding expected outcome of a game based on the Glicko rating system formula:
# http://www.glicko.net/glicko/glicko.pdf p.5

def glicko_p(MMR1, MMR2):
    MMR1 = np.array(MMR1)
    MMR2 = np.array(MMR2)
    RD = np.sqrt(12800)
    q = np.log(10)/400
    g = 1 / (np.sqrt(1 + (np.prod([3, np.power(q, 2), np.power(RD, 2)])) / np.power(np.pi, 2)))
    E_p1 = 1 / (1 + np.power(10, np.prod([-g,(MMR1-MMR2),1/400])))
    E_p2 = 1 - E_p1
    return E_p1, E_p2

In [8]:
grouped = pd.DataFrame(df.groupby('_id')[['battleTag', 'mmr.rating']].agg(list)) #groupby id and get both players rating
grouped 

,battleTag,mmr.rating
_id,,
616187c02cbca25ad0b98eb2,"[Flexitus#21593, czubek#1463]","[1335.201771887519, 1332.138809199506]"
616188382cbca25ad0b9a6d3,"[SPXVranke#2480, Papitsch#21830]","[1731.976186646819, 1407.204703497218]"
616188a12cbca25ad0b9baab,"[WorldEdit#2941, Kanaxis#21873]","[1584.703714657364, 1564.1649170873827]"
616189242cbca25ad0b9d6c0,"[EdoBoi#2276, niluje#21538]","[2014.5010124505268, 1269.4239586480169]"
616189272cbca25ad0b9d752,"[Arx#1383, Delta#23312]","[1611.4977820144486, 1581.8456381438014]"
...,...,...
61cb00bf47a154ba1c43a524,"[Kenny#26603, Chaganaga#2391]","[1672.911336207745, 1650.344094170974]"
61cb012447a154ba1c43b949,"[Alchemist#31554, VladoS23#21572]","[1163.7420001167225, 1039.469040940771]"
61cb013747a154ba1c43bdfb,"[Alexandr#21108, Sapphire#2194]","[1262.8008085645497, 1262.1421944668618]"


In [9]:
grouped['MMR1'] = pd.DataFrame(grouped["mmr.rating"].to_list(), columns=['MMR1', 'MMR2'], index=grouped.index)['MMR1']
grouped['MMR2'] = pd.DataFrame(grouped["mmr.rating"].to_list(), columns=['MMR1', 'MMR2'], index=grouped.index)['MMR2']
grouped['p1'] = pd.DataFrame(grouped["battleTag"].to_list(), columns=['p1', 'p2'], index=grouped.index)['p1']
grouped['p2'] = pd.DataFrame(grouped["battleTag"].to_list(), columns=['p1', 'p2'], index=grouped.index)['p2']

In [10]:
grouped['pwin1'], grouped['pwin2'] = glicko_p(grouped['MMR1'], grouped['MMR2'])

C:\Users\Nikos\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [12]:
grouped

,battleTag,mmr.rating,MMR1,MMR2,p1,p2,pwin1,pwin2
_id,,,,,,,,
616187c02cbca25ad0b98eb2,"[Flexitus#21593, czubek#1463]","[1335.201771887519, 1332.138809199506]",1335.201772,1332.138809,Flexitus#21593,czubek#1463,0.504149,0.495851
616188382cbca25ad0b9a6d3,"[SPXVranke#2480, Papitsch#21830]","[1731.976186646819, 1407.204703497218]",1731.976187,1407.204703,SPXVranke#2480,Papitsch#21830,0.853153,0.146847
616188a12cbca25ad0b9baab,"[WorldEdit#2941, Kanaxis#21873]","[1584.703714657364, 1564.1649170873827]",1584.703715,1564.164917,WorldEdit#2941,Kanaxis#21873,0.527790,0.472210
616189242cbca25ad0b9d6c0,"[EdoBoi#2276, niluje#21538]","[2014.5010124505268, 1269.4239586480169]",2014.501012,1269.423959,EdoBoi#2276,niluje#21538,0.982650,0.017350
616189272cbca25ad0b9d752,"[Arx#1383, Delta#23312]","[1611.4977820144486, 1581.8456381438014]",1611.497782,1581.845638,Arx#1383,Delta#23312,0.540076,0.459924
...,...,...,...,...,...,...,...,...
61cb00bf47a154ba1c43a524,"[Kenny#26603, Chaganaga#2391]","[1672.911336207745, 1650.344094170974]",1672.911336,1650.344094,Kenny#26603,Chaganaga#2391,0.530528,0.469472
61cb012447a154ba1c43b949,"[Alchemist#31554, VladoS23#21572]","[1163.7420001167225, 1039.469040940771]",1163.742000,1039.469041,Alchemist#31554,VladoS23#21572,0.662239,0.337761
61cb013747a154ba1c43bdfb,"[Alexandr#21108, Sapphire#2194]","[1262.8008085645497, 1262.1421944668618]",1262.800809,1262.142194,Alexandr#21108,Sapphire#2194,0.500892,0.499108


In [13]:
grouped1 = grouped.reset_index().melt(id_vars = ['_id'], value_vars=['pwin1', 'pwin2']).sort_values(['_id', 'variable'])
grouped2 = grouped.reset_index().melt(id_vars = ['_id'], value_vars=['p1', 'p2']).sort_values(['_id', 'variable'])

In [14]:
grouped1

,_id,variable,value
0,616187c02cbca25ad0b98eb2,pwin1,0.504149
201620,616187c02cbca25ad0b98eb2,pwin2,0.495851
1,616188382cbca25ad0b9a6d3,pwin1,0.853153
201621,616188382cbca25ad0b9a6d3,pwin2,0.146847
2,616188a12cbca25ad0b9baab,pwin1,0.527790
...,...,...,...
403237,61cb013747a154ba1c43bdfb,pwin2,0.499108
201618,61cb014147a154ba1c43c098,pwin1,0.540090
403238,61cb014147a154ba1c43c098,pwin2,0.459910
201619,61cb01a647a154ba1c43d75e,pwin1,0.539893


In [15]:
grouped2

,_id,variable,value
0,616187c02cbca25ad0b98eb2,p1,Flexitus#21593
201620,616187c02cbca25ad0b98eb2,p2,czubek#1463
1,616188382cbca25ad0b9a6d3,p1,SPXVranke#2480
201621,616188382cbca25ad0b9a6d3,p2,Papitsch#21830
2,616188a12cbca25ad0b9baab,p1,WorldEdit#2941
...,...,...,...
403237,61cb013747a154ba1c43bdfb,p2,Sapphire#2194
201618,61cb014147a154ba1c43c098,p1,GicKu#2820
403238,61cb014147a154ba1c43c098,p2,ParisBoris#2218
201619,61cb01a647a154ba1c43d75e,p1,HIPPO#31791


In [16]:
grouped1['battleTag'] = grouped2['value']

In [17]:
grouped1

,_id,variable,value,battleTag
0,616187c02cbca25ad0b98eb2,pwin1,0.504149,Flexitus#21593
201620,616187c02cbca25ad0b98eb2,pwin2,0.495851,czubek#1463
1,616188382cbca25ad0b9a6d3,pwin1,0.853153,SPXVranke#2480
201621,616188382cbca25ad0b9a6d3,pwin2,0.146847,Papitsch#21830
2,616188a12cbca25ad0b9baab,pwin1,0.527790,WorldEdit#2941
...,...,...,...,...
403237,61cb013747a154ba1c43bdfb,pwin2,0.499108,Sapphire#2194
201618,61cb014147a154ba1c43c098,pwin1,0.540090,GicKu#2820
403238,61cb014147a154ba1c43c098,pwin2,0.459910,ParisBoris#2218
201619,61cb01a647a154ba1c43d75e,pwin1,0.539893,HIPPO#31791


In [18]:
grouped1 = grouped1.reset_index()

In [19]:
grouped1

,index,_id,variable,value,battleTag
0,0,616187c02cbca25ad0b98eb2,pwin1,0.504149,Flexitus#21593
1,201620,616187c02cbca25ad0b98eb2,pwin2,0.495851,czubek#1463
2,1,616188382cbca25ad0b9a6d3,pwin1,0.853153,SPXVranke#2480
3,201621,616188382cbca25ad0b9a6d3,pwin2,0.146847,Papitsch#21830
4,2,616188a12cbca25ad0b9baab,pwin1,0.527790,WorldEdit#2941
...,...,...,...,...,...
403235,403237,61cb013747a154ba1c43bdfb,pwin2,0.499108,Sapphire#2194
403236,201618,61cb014147a154ba1c43c098,pwin1,0.540090,GicKu#2820
403237,403238,61cb014147a154ba1c43c098,pwin2,0.459910,ParisBoris#2218
403238,201619,61cb01a647a154ba1c43d75e,pwin1,0.539893,HIPPO#31791


In [20]:
df = df.merge(grouped1, on=['_id','battleTag'])

In [21]:
df

,Unnamed: 0,raceId,battleTag,overallScore.UNIT_SCORE,overallScore.HERO_SCORE,overallScore.RESOURCE_SCORE,overallScore.TOTAL_SCORE,unitScore.UNITS_PRODUCED,unitScore.UNITS_KILLED,unitScore.STRUCTURES_PRODUCED,...,Hero2,Hero2-lvl,Hero3,Hero3-lvl,hero_combo,nheroes,duration,index,variable,value
0,0,4,Flexitus#21593,9240,6580,6255,22075,26,15,9,...,NaN,NaN,NaN,NaN,potm--,1,8.083333,0,pwin1,0.504149
1,1,4,czubek#1463,7910,2670,5205,15785,30,6,9,...,NaN,NaN,NaN,NaN,dh--,1,8.083333,201620,pwin2,0.495851
2,2,3,SPXVranke#2480,6935,5120,3988,16043,23,15,11,...,NaN,NaN,NaN,NaN,lich--,1,5.633333,1,pwin1,0.853153
3,3,3,Papitsch#21830,4710,1560,3160,9430,29,5,8,...,NaN,NaN,NaN,NaN,cryptlord--,1,5.633333,201621,pwin2,0.146847
4,4,4,WorldEdit#2941,20505,23020,10475,54000,49,47,13,...,pitlord,3.0,darkranger,2.0,kotg-pitlord-darkranger,3,11.900000,2,pwin1,0.527790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403235,403235,3,Sapphire#2194,28905,37460,15126,81491,76,64,15,...,lich,4.0,darkranger,4.0,dk-lich-darkranger,3,20.616667,403237,pwin2,0.499108
403236,403236,3,GicKu#2820,45660,48200,24548,118408,68,96,43,...,lich,5.0,cryptlord,4.0,dk-lich-cryptlord,3,27.866667,201618,pwin1,0.540090
403237,403237,2,ParisBoris#2218,32955,20140,33666,86761,58,56,28,...,tauren,3.0,sh,1.0,panda-tauren-sh,3,27.866667,403238,pwin2,0.459910
403238,403238,1,HIPPO#31791,30630,24890,18287,73807,89,48,34,...,mk,4.0,paladin,1.0,am-mk-paladin,3,23.783333,201619,pwin1,0.539893


In [22]:
df.drop(['Unnamed: 0', 'index', 'variable'], axis=1, inplace=True)

In [23]:
df.rename({'value': 'pwin'}, axis=1, inplace=True)

In [26]:
df[0:1].T

,0
raceId,4
battleTag,Flexitus#21593
overallScore.UNIT_SCORE,9240
overallScore.HERO_SCORE,6580
overallScore.RESOURCE_SCORE,6255
overallScore.TOTAL_SCORE,22075
unitScore.UNITS_PRODUCED,26
unitScore.UNITS_KILLED,15
unitScore.STRUCTURES_PRODUCED,9
unitScore.STRUCTURES_RAZED,2


In [25]:
df.to_csv('s9pwin.csv')

In [2]:
df = pd.read_csv('s9pwin.csv')

In [9]:
#race

# 0 == random
# 1 == hum
# 2 == orc
# 4 == elf
# 8 == ud

#raceId

# 1 == hum
# 2 == orc
# 3 = ud
# 4 == elf

# initial unit & total score bonuses

# 1005, 965, 945, and 825 for Undead, Human, Orc, and NE, respectively

# subtract initial unit score & total score bonuses from players' scores

df.loc[df['raceId'].eq(1), ['overallScore.UNIT_SCORE', 'overallScore.TOTAL_SCORE']] -= 965 #hum
df.loc[df['raceId'].eq(2), ['overallScore.UNIT_SCORE', 'overallScore.TOTAL_SCORE']] -= 945 #orc
df.loc[df['raceId'].eq(3), ['overallScore.UNIT_SCORE', 'overallScore.TOTAL_SCORE']] -= 1005 #ud
df.loc[df['raceId'].eq(4), ['overallScore.UNIT_SCORE', 'overallScore.TOTAL_SCORE']] -= 825 #NE

In [10]:
df

,Unnamed: 0,raceId,battleTag,overallScore.UNIT_SCORE,overallScore.HERO_SCORE,overallScore.RESOURCE_SCORE,overallScore.TOTAL_SCORE,unitScore.UNITS_PRODUCED,unitScore.UNITS_KILLED,unitScore.STRUCTURES_PRODUCED,...,Hero1,Hero1-lvl,Hero2,Hero2-lvl,Hero3,Hero3-lvl,hero_combo,nheroes,duration,pwin
0,0,4,Flexitus#21593,8415,6580,6255,21250,26,15,9,...,potm,3,NaN,NaN,NaN,NaN,potm--,1,8.083333,0.504149
1,1,4,czubek#1463,7085,2670,5205,14960,30,6,9,...,dh,2,NaN,NaN,NaN,NaN,dh--,1,8.083333,0.495851
2,2,3,SPXVranke#2480,5930,5120,3988,15038,23,15,11,...,lich,3,NaN,NaN,NaN,NaN,lich--,1,5.633333,0.853153
3,3,3,Papitsch#21830,3705,1560,3160,8425,29,5,8,...,cryptlord,1,NaN,NaN,NaN,NaN,cryptlord--,1,5.633333,0.146847
4,4,4,WorldEdit#2941,19680,23020,10475,53175,49,47,13,...,kotg,4,pitlord,3.0,darkranger,2.0,kotg-pitlord-darkranger,3,11.900000,0.527790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403235,403235,3,Sapphire#2194,27900,37460,15126,80486,76,64,15,...,dk,5,lich,4.0,darkranger,4.0,dk-lich-darkranger,3,20.616667,0.499108
403236,403236,3,GicKu#2820,44655,48200,24548,117403,68,96,43,...,dk,6,lich,5.0,cryptlord,4.0,dk-lich-cryptlord,3,27.866667,0.540090
403237,403237,2,ParisBoris#2218,32010,20140,33666,85816,58,56,28,...,panda,4,tauren,3.0,sh,1.0,panda-tauren-sh,3,27.866667,0.459910
403238,403238,1,HIPPO#31791,29665,24890,18287,72842,89,48,34,...,am,4,mk,4.0,paladin,1.0,am-mk-paladin,3,23.783333,0.539893


In [12]:
# insert opponent scores

to_reverse = df[['_id', 'overallScore.UNIT_SCORE', 'overallScore.HERO_SCORE', 'overallScore.RESOURCE_SCORE',
                'overallScore.TOTAL_SCORE', 'unitScore.UNITS_PRODUCED', 'unitScore.UNITS_KILLED',
                'unitScore.STRUCTURES_PRODUCED', 'unitScore.STRUCTURES_RAZED', 'unitScore.LARGEST_ARMY',
                'heroScore.HEROES_KILLED', 'heroScore.ITEMS_OBTAINED', 'heroScore.MERCS_HIRED',
                'heroScore.EXP_GAINED', 'resourceScore.GOLD_COLLECTED', 'resourceScore.LUMBER_COLLECTED',
                'resourceScore.TECH_PERCENTAGE', 'resourceScore.GOLD_UPKEEP_LOST', 'raceId',
                'mmr.rating', 'pings']]


def reversing(x):
    x[['overallScore.UNIT_SCORE', 'overallScore.HERO_SCORE', 'overallScore.RESOURCE_SCORE',
                'overallScore.TOTAL_SCORE', 'unitScore.UNITS_PRODUCED', 'unitScore.UNITS_KILLED',
                'unitScore.STRUCTURES_PRODUCED', 'unitScore.STRUCTURES_RAZED', 'unitScore.LARGEST_ARMY',
                'heroScore.HEROES_KILLED', 'heroScore.ITEMS_OBTAINED', 'heroScore.MERCS_HIRED',
                'heroScore.EXP_GAINED', 'resourceScore.GOLD_COLLECTED', 'resourceScore.LUMBER_COLLECTED',
                'resourceScore.TECH_PERCENTAGE', 'resourceScore.GOLD_UPKEEP_LOST', 'raceId',
                'mmr.rating', 'pings']] = x[['overallScore.UNIT_SCORE', 'overallScore.HERO_SCORE', 'overallScore.RESOURCE_SCORE',
                'overallScore.TOTAL_SCORE', 'unitScore.UNITS_PRODUCED', 'unitScore.UNITS_KILLED',
                'unitScore.STRUCTURES_PRODUCED', 'unitScore.STRUCTURES_RAZED', 'unitScore.LARGEST_ARMY',
                'heroScore.HEROES_KILLED', 'heroScore.ITEMS_OBTAINED', 'heroScore.MERCS_HIRED',
                'heroScore.EXP_GAINED', 'resourceScore.GOLD_COLLECTED', 'resourceScore.LUMBER_COLLECTED',
                'resourceScore.TECH_PERCENTAGE', 'resourceScore.GOLD_UPKEEP_LOST', 'raceId',
                'mmr.rating', 'pings']].iloc[::-1].values
    return x



In [13]:
dfg2 = to_reverse.groupby('_id', sort=False).apply(reversing)

In [14]:
dfg2

,_id,overallScore.UNIT_SCORE,overallScore.HERO_SCORE,overallScore.RESOURCE_SCORE,overallScore.TOTAL_SCORE,unitScore.UNITS_PRODUCED,unitScore.UNITS_KILLED,unitScore.STRUCTURES_PRODUCED,unitScore.STRUCTURES_RAZED,unitScore.LARGEST_ARMY,...,heroScore.ITEMS_OBTAINED,heroScore.MERCS_HIRED,heroScore.EXP_GAINED,resourceScore.GOLD_COLLECTED,resourceScore.LUMBER_COLLECTED,resourceScore.TECH_PERCENTAGE,resourceScore.GOLD_UPKEEP_LOST,raceId,mmr.rating,pings
0,616187c02cbca25ad0b98eb2,7085.0,2670.0,5205.0,14960.0,30.0,6.0,9.0,0.0,30.0,...,2.0,2.0,267.0,4430.0,775.0,0.0,0.0,4.0,1332.138809,41.0
1,616187c02cbca25ad0b98eb2,8415.0,6580.0,6255.0,21250.0,26.0,15.0,9.0,2.0,50.0,...,2.0,0.0,658.0,4630.0,1625.0,0.0,0.0,4.0,1335.201772,41.0
2,616188382cbca25ad0b9a6d3,3705.0,1560.0,3160.0,8425.0,29.0,5.0,8.0,0.0,23.0,...,2.0,0.0,156.0,2610.0,550.0,0.0,0.0,3.0,1407.204703,74.0
3,616188382cbca25ad0b9a6d3,5930.0,5120.0,3988.0,15038.0,23.0,15.0,11.0,0.0,30.0,...,2.0,0.0,512.0,3140.0,848.0,0.0,0.0,3.0,1731.976187,57.0
4,616188a12cbca25ad0b9baab,19140.0,16950.0,9346.0,45436.0,56.0,46.0,13.0,0.0,48.0,...,18.0,0.0,1695.0,6810.0,2536.0,13.0,0.0,2.0,1564.164917,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403235,61cb013747a154ba1c43bdfb,22460.0,17750.0,17961.0,58171.0,64.0,44.0,21.0,0.0,74.0,...,15.0,0.0,1775.0,13265.0,4696.0,14.0,1275.0,3.0,1262.800809,26.0
403236,61cb014147a154ba1c43c098,32010.0,20140.0,33666.0,85816.0,58.0,56.0,28.0,8.0,82.0,...,60.0,9.0,2014.0,24079.0,9587.0,37.0,2601.0,2.0,1233.523681,16.0
403237,61cb014147a154ba1c43c098,44655.0,48200.0,24548.0,117403.0,68.0,96.0,43.0,7.0,76.0,...,33.0,3.0,4820.0,18348.0,6200.0,35.0,4212.0,3.0,1263.186351,17.0
403238,61cb01a647a154ba1c43d75e,23195.0,31960.0,17506.0,72661.0,37.0,62.0,25.0,13.0,55.0,...,13.0,0.0,3196.0,11564.0,5942.0,40.0,936.0,1.0,1388.678496,119.0


In [15]:
df[['e.overallScore.UNIT_SCORE', 'e.overallScore.HERO_SCORE', 'e.overallScore.RESOURCE_SCORE',
                'e.overallScore.TOTAL_SCORE', 'e.unitScore.UNITS_PRODUCED', 'e.unitScore.UNITS_KILLED',
                'e.unitScore.STRUCTURES_PRODUCED', 'e.unitScore.STRUCTURES_RAZED', 'e.unitScore.LARGEST_ARMY',
                'e.heroScore.HEROES_KILLED', 'e.heroScore.ITEMS_OBTAINED', 'e.heroScore.MERCS_HIRED',
                'e.heroScore.EXP_GAINED', 'e.resourceScore.GOLD_COLLECTED', 'e.resourceScore.LUMBER_COLLECTED',
                'e.resourceScore.TECH_PERCENTAGE', 'e.resourceScore.GOLD_UPKEEP_LOST', 'e.raceId',
                'e.mmr.rating', 'e.pings']] = dfg2[['overallScore.UNIT_SCORE', 'overallScore.HERO_SCORE', 'overallScore.RESOURCE_SCORE',
                'overallScore.TOTAL_SCORE', 'unitScore.UNITS_PRODUCED', 'unitScore.UNITS_KILLED',
                'unitScore.STRUCTURES_PRODUCED', 'unitScore.STRUCTURES_RAZED', 'unitScore.LARGEST_ARMY',
                'heroScore.HEROES_KILLED', 'heroScore.ITEMS_OBTAINED', 'heroScore.MERCS_HIRED',
                'heroScore.EXP_GAINED', 'resourceScore.GOLD_COLLECTED', 'resourceScore.LUMBER_COLLECTED',
                'resourceScore.TECH_PERCENTAGE', 'resourceScore.GOLD_UPKEEP_LOST', 'raceId',
                'mmr.rating', 'pings']]

In [25]:
df

,Unnamed: 0,raceId,battleTag,overallScore.UNIT_SCORE,overallScore.HERO_SCORE,overallScore.RESOURCE_SCORE,overallScore.TOTAL_SCORE,unitScore.UNITS_PRODUCED,unitScore.UNITS_KILLED,unitScore.STRUCTURES_PRODUCED,...,e.heroScore.ITEMS_OBTAINED,e.heroScore.MERCS_HIRED,e.heroScore.EXP_GAINED,e.resourceScore.GOLD_COLLECTED,e.resourceScore.LUMBER_COLLECTED,e.resourceScore.TECH_PERCENTAGE,e.resourceScore.GOLD_UPKEEP_LOST,e.raceId,e.mmr.rating,e.pings
0,0,4,Flexitus#21593,8415,6580,6255,21250,26,15,9,...,2.0,2.0,267.0,4430.0,775.0,0.0,0.0,4.0,1332.138809,41.0
1,1,4,czubek#1463,7085,2670,5205,14960,30,6,9,...,2.0,0.0,658.0,4630.0,1625.0,0.0,0.0,4.0,1335.201772,41.0
2,2,3,SPXVranke#2480,5930,5120,3988,15038,23,15,11,...,2.0,0.0,156.0,2610.0,550.0,0.0,0.0,3.0,1407.204703,74.0
3,3,3,Papitsch#21830,3705,1560,3160,8425,29,5,8,...,2.0,0.0,512.0,3140.0,848.0,0.0,0.0,3.0,1731.976187,57.0
4,4,4,WorldEdit#2941,19680,23020,10475,53175,49,47,13,...,18.0,0.0,1695.0,6810.0,2536.0,13.0,0.0,2.0,1564.164917,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403235,403235,3,Sapphire#2194,27900,37460,15126,80486,76,64,15,...,15.0,0.0,1775.0,13265.0,4696.0,14.0,1275.0,3.0,1262.800809,26.0
403236,403236,3,GicKu#2820,44655,48200,24548,117403,68,96,43,...,60.0,9.0,2014.0,24079.0,9587.0,37.0,2601.0,2.0,1233.523681,16.0
403237,403237,2,ParisBoris#2218,32010,20140,33666,85816,58,56,28,...,33.0,3.0,4820.0,18348.0,6200.0,35.0,4212.0,3.0,1263.186351,17.0
403238,403238,1,HIPPO#31791,29665,24890,18287,72842,89,48,34,...,13.0,0.0,3196.0,11564.0,5942.0,40.0,936.0,1.0,1388.678496,119.0


In [17]:
df.to_csv('s9_final_vs.csv')

In [3]:
df = pd.read_csv('s9_final_vs.csv')

In [28]:
(df['overallScore.TOTAL_SCORE'] - df['e.overallScore.TOTAL_SCORE']) / df['e.overallScore.TOTAL_SCORE']

0         0.420455
1        -0.296000
2         0.784926
3        -0.439753
4         0.170327
            ...   
403235    0.383610
403236    0.368078
403237   -0.269048
403238    0.002491
403239   -0.002485
Length: 403240, dtype: float64